In [ ]:
"""
This script sets up a pipeline for evaluating a language model using the "Kubermatic/cncf-question-and-answer-dataset-for-llm-training" dataset.
It leverages advanced techniques including 4-bit quantization and evaluates the model using BLEU score.

Dependencies:
- accelerate
- datasets
- trl
- peft
- bitsandbytes
- evaluate
- transformers
- torch
- tqdm
- pandas
- huggingface_hub

Environment Setup:
- Ensure Hugging Face Hub authentication with 'notebook_login'.

Model Configuration:
- Uses BitsAndBytesConfig for 4-bit quantization.
- Configures AutoModelForCausalLM for causal language modeling tasks.

Training Data:
- Loads a subset of 1000 samples from the dataset.

Example Question/Answer:
- Tokenizes and generates an answer for a sample question from the dataset.
- Evaluates the generated answer against the reference answer using BLEU score.

Output:
- Prints the example question, reference answer, predicted answer, and BLEU score.
"""

!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U peft
!pip install -q -U -i https://pypi.org/simple/ bitsandbytes
!pip install evaluate -q -U

import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from datasets import load_dataset
import torch
import re
from tqdm import tqdm
import pandas as pd
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
import accelerate
from trl import SFTTrainer
from transformers import (AutoModelForCausalLM,
                          AutoModelForQuestionAnswering,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          )

In [ ]:
from huggingface_hub import hf_hub_download
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_id = "google/gemma-1.1-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
# TODO: Check if this can be changed to AutoModelForQuestionAnswering with GEMMA
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# Training Data
dataset = load_dataset("Kubermatic/cncf-question-and-answer-dataset-for-llm-training", split="train[:1000]")

In [ ]:
#example question/answer

question = dataset[1]["Question"]
device = "cuda:0"
inputs = tokenizer(question, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=500)
tokenizer.decode(outputs[0], skip_special_tokens=True)

prediction = [tokenizer.decode(outputs[0], skip_special_tokens=True)]
reference = dataset[1]["Answer"]
reference = [reference]

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=prediction, references=reference)
print("Question from dataset: " + question)
print("Answer from dataset: " + reference[0])
print("Predicted Answer: " + prediction[0])
print(results)